In [1]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tltk
# tltk.pos_load()
%matplotlib inline

In [2]:
# Path to the preprocessed data
best_processed_path = 'w_review_train.csv'

In [152]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
print(CHARS_MAP)
print(len(CHARS_MAP))

{'*': 11, 'other': 80, 'Z': 59, '!': 2, '๐': 165, '.': 15, 'ื': 147, 'ภ': 125, 'บ': 119, ' ': 1, 'h': 72, 'k': 75, '>': 31, '$': 5, '\n': 0, '’': 176, 'J': 43, '7': 24, 'พ': 123, 'L': 45, 'M': 46, 'จ': 101, ':': 27, 'น': 118, 'E': 38, '0': 17, '@': 33, 'ฆ': 99, '?': 32, 'O': 48, 't': 85, 'ฑ': 110, 'j': 74, 'D': 37, 's': 84, '"': 3, ')': 10, 'ป': 120, '๕': 170, 'โ': 153, 'เ': 151, 'I': 42, 'ฉ': 102, '๖': 171, 'H': 41, 'w': 88, '1': 18, 'B': 35, 'z': 91, 'ด': 113, '\ufeff': 177, '๙': 174, 'ง': 100, '/': 16, 'อ': 137, 'ึ': 146, 'G': 40, 'ฯ': 139, 'U': 54, 'u': 86, '8': 25, 'ร': 128, ']': 62, '้': 160, 'K': 44, '์': 163, '6': 23, 'ใ': 154, 'ฅ': 98, 'c': 67, 'W': 56, 'b': 66, 'ฃ': 96, 'แ': 152, 'f': 70, '#': 4, 'ท': 116, "'": 8, 'i': 73, 'ี': 145, '&': 7, ';': 28, 'ช': 103, 'C': 36, '๔': 169, 'ฏ': 108, '๓': 168, '9': 26, 'x': 89, 'ฺ': 150, 'X': 57, 'ย': 127, 'ษ': 133, '๘': 173, '+': 12, 'r': 83, 'ฝ': 122, '5': 22, 'ๆ': 157, 'a': 65, '3': 20, '^': 63, 'ำ': 143, 'ิ': 144, 'ส': 134, '๋': 162, 

In [3]:
from keras.models import Model
from keras.layers import Embedding,Dense, Conv1D, Flatten, TimeDistributed, Dropout
from keras.layers import Input, GRU, Bidirectional
from keras.optimizers import Adam
# def get_my_best_model2():
#     input1 = Input(shape=(21,))
#     x = Embedding(178,8)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = TimeDistributed(Dense(5))(x)ฅ
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='binary_crossentropy',
#                  metrics=['acc'])          
#     return model

def get_my_best_model():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])          
    return model
    

Using TensorFlow backend.


In [4]:
weight_path_my_best_model2='model_best.h5'
my_best_model2 = get_my_best_model()
my_best_model2.load_weights(weight_path_my_best_model2)
#my_best_model2.make_predict_function()
my_best_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 21, 8)             1424      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 21, 100)           4100      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 5)             505       
_________________________________________________________________
flatten_1 (Flatten)          (None, 105)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10600     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
__________

In [7]:
df = pd.read_csv(best_processed_path, sep=';', header=None)
df2 = pd.read_csv("test_file.csv", sep=';', header=None)

In [8]:
test_set = df2[1][1:]
test_set

1            ร้านนี้จะอยู่เส้นสันกำแพง-แม่ออน เลยแยกบ่...
2       สั่งไป2 เมนู คือมัชฉะลาเต้ร้อน กับ ไอศครีมชาเข...
3       ครัววงเดือน  \n\nหิวดึกๆ ตระเวนหาร้านทาน มาเจอ...
4            จะว่าเป็นเจ้าประจำก็คงไม่ผิด แต่ก็ไม่กล้า...
5       ถ้าคิดถึงสลัดผมคิดถึงร้านนี้เป็นร้านแรกๆเลยครั...
6       กระบี่ก็ต้องไม่พลาด ร้านนี้ค่ะ ร้านปูดำ ส่วนอา...
7       ร้านดัง  ของหวานอร่อยที่สุดก็คงต้องยกให้  afte...
8       อาหารหลากหลายไทย จีน ฝรั่ง ชาบูก็มา ของหวานมีฮ...
9       เป็นร้านอาหารเพื่อสุขภาพที่รสชาติดี ทานแล้วไม่...
10      ดีใจที่จุดพักรถมอร์เตอร์เวย์ขาเข้า มีร้านอาหาร...
11      ร้านชาบูเล็กๆ ราคาเบาๆ แต่คุณภาพคับแก้ว shabuy...
12      ร้านนี้เป็นร้านกระทะร้อนโด่งดังในย่านเยาวราช ม...
13        》 มาธุระแถวพืชสวนโลก เกินเวลาทานมื้อเที่ยง เ...
14      เป็นร้านกาแฟอยู่ในร้านอาหารเสน่ห์ดอยหลวง \nร้า...
15      เป็นร้านข้าวซอยที่อยู่แถวสี่แยกไปอำเภอปาย ร้าน...
16      หาร้านทานตอนกลางวันวันเสาร์แบบด่วนๆ จองร้านนี้...
17      ร้านปิ้งย่างบุฟเฟห์เกาหลี โดยคนเกาหลี กับร้าน ...
18      ร้านนี

In [9]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df[0].shift(i + 1)
      df['char{}'.format(i+1)] = df[0].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [10]:
word_test = [x for x in df[0][1]]
print(''.join(word_test))

อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา ตัวร้านตั้งอยู่ที่ถนนพุทธมณฑลสาย 3 ไปตาม ถ.ปิ่นเกล้า-นครชัยศรี เมื่อถึงพุทธมณฑลสาย 3 ก็เลี้ยวเข้าไปประมาณ 500 เมตร ร้านอยู่ทางซ้ายมือค่ะ มีคนบอกมาว่าความพิเศษของร้านนี้คือกุ๊กเก่าและเป็นกุ๊กรุ่นสุดท้ายจาก "ฮก ลก ซิ่ว” ภัตตาคารจีนชื่อดังย่านราชประสงค์ ที่เลิกกิจการไปแล้ว ต้องคนที่อายุเลข 5 ขึ้นไปจึงจะเคยกิน ฮก ลก ซิ่ว  จานเด็ดที่มีขายที่นี่แห่งเดียวในเมืองไทยคือ ปลาเต๋าเต้ย 2 ฤดู เป็นสูตรจากมาเลเซีย นอกนั้นก็มี ผัดผักน้ำมันหอย ไก่เบตง เคาหยก ปูทะเลซุปน้ำใสหม้อไฟ เต้าหู้แคระยัดไส้หม้อดิน และ ลูกชิ้นแคระ 
อาหารที่เราแนะนำคือไก่เบตง (คล้ายๆไก่แช่เหล้า) เสริฟพร้อมกับหอมเจียว และน้ำจิ้มน้ำพริกเผาสูตรเด็ดของทางร้าน 
เมนูข้าวผัดหนำเลียบก็อร่อยค่ะ ชอบมากๆ


In [11]:
word_test_df2 = pd.DataFrame(word_test)
#print(word_test_df2)

n_pad = 21
n_pad_2 = int((n_pad - 1)/2)
pad = [{0: ' '}]
df_pad = pd.DataFrame(pad * n_pad_2)
#print(df_pad)

word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
#print(word_test_df2[0][:21])
word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))

NameError: name 'CHARS_MAP' is not defined

In [12]:
df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

# convert pandas dataframe to numpy array to feed to the model
x_char = df_with_context[char_row].as_matrix()

In [13]:
y_pred = my_best_model2.predict(x_char)

prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

ValueError: could not convert string to float: 'ก'

In [14]:
print(len(y_pred))
print(len(word_test))
word_token = []
word_tmp = ''
for i in range(len(y_pred)):
    if(y_pred[i] == 1):
        word_token.append(word_tmp)
        word_tmp = ''
    word_tmp += word_test[i]
else:
    word_token.append(word_tmp)
word_token = word_token[1:]
print(word_token)

NameError: name 'y_pred' is not defined

In [15]:
def cutKumKrub(string):
    word_test = [x for x in string]
    word_test_df2 = pd.DataFrame(word_test)

    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{0: ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

    word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
    word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))
    
    df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
                 ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()
    
    y_pred = my_best_model2.predict(x_char)

    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
    word_token = []
    word_tmp = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            word_token.append(word_tmp)
            word_tmp = ''
        word_tmp += word_test[i]
    else:
        word_token.append(word_tmp)
    word_token = word_token[1:]
    return word_token

In [16]:
def mergeLabel(review, rating):
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
    for i in range(len(cleaned)):
        cleaned[i] = cleaned[i].replace("\n", "")
        cleaned[i] = cleaned[i].replace("(", "")
        cleaned[i] = cleaned[i].replace(")", "")
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))
    label = [int(rating)] * len(cleaned)
    training_data = list(zip(cleaned, label))
    return training_data

In [17]:
mergeLabel(df[0][1], df[1][1])

NameError: name 'CHARS_MAP' is not defined

In [18]:
demo = df[:][:40000]
demo

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [19]:
train = []
for i in range(len(demo)):
    train.append((demo[0][i], demo[1][i]))

In [168]:
# train

In [20]:
import re
def remove_htmlTags(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
    
def isValidWord(text):
    dic = {text[0]:1}
    for c in text[1:]:
        if c not in dic:
            return True
        else:
            dic[c] += 1
    if dic[text[0]] > 1:
        return False
    else:
        return True
        
def tokenize(review):
    
#     tags = tltk.nlp.pos_tag(review.replace("\n", ""))
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, noSpace))

#     # Filter parts of speech of text
#     tags = tltk.nlp.pos_tag(remove_htmlTags(review.replace("\n", "")))
#     finalWord = []
#     filteredTags = {'NOUN', 'VERB', 'ADJ','ADV', 'AUX', 'PART'}
#     for sentence in tags:
#         for word, tag in sentence:
#             if tag in filteredTags:
#                 if len(word) > 1 and isValidWord(word):
#                     finalWord.append(remove_htmlTags(word))
    return finalWord

# tokenize(train[0][0])
# train[0][0]

In [170]:
# from itertools import chain
# vocabulary = set(chain(*[tokenize(i[0].lower()) for i in train]))

In [21]:

tokenize(test_set.iloc[0])


NameError: name 'CHARS_MAP' is not defined

In [23]:
from IPython.display import clear_output
corpus_test = []
for i in range(len(test_set)):
    print(str(i+1) + "/" + str(len(train)))
    corpus_test.append(tokenize(test_set.iloc[i]))

# corpus

1/40000


NameError: name 'CHARS_MAP' is not defined

In [22]:

import pickle


# pickle_out = open("corpus3.pickle","wb")
# pickle.dump(corpus, pickle_out)
# pickle_out.close()

pickle_out = open("corpus_test.pickle","wb")
pickle.dump(corpus_test, pickle_out)
pickle_out.close()


corpus = corpus[:40000]
len(corpus)

NameError: name 'corpus_test' is not defined

In [24]:
pickle_in = open("corpus4.pickle","rb")
corpus = pickle.load(pickle_in)
len(corpus)
# corpus

40000

In [25]:
# corpus
# corpus2 = corpus[:1000]
newcor = corpus
test = corpus_test
# tltk.nlp.pos_tag(train[0][0])
len(test)

0

In [26]:
def mytok(mylist):
    for i in range(len(mylist)):
        mylist[i] = mylist[i].lower()
    return mylist
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=mytok, binary = True)
tfidf_matrix = tfidf.fit_transform(newcor)

In [27]:
tfidf_matrix

<40000x167678 sparse matrix of type '<class 'numpy.float64'>'
	with 3585680 stored elements in Compressed Sparse Row format>

In [179]:

# idf = tfidf.idf_
# print (tfidf.get_feature_names(), idf)

In [180]:
# corpus = [
#     tokenize(df[0][0]),
#     tokenize(df[0][1]),
#     tokenize(df[0][2])
# ]

In [36]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# pca = PCA(n_components = 300)

# x_train = pca.fit_transform(x_train)
svd = TruncatedSVD(n_components=300, n_iter=10, random_state=42)
x_train = svd.fit_transform(tfidf_matrix)

y_train = []

for i in range(40000):
    y_train.append(train[i][1])
y_train = np.array(y_train)

/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


KeyboardInterrupt: 

In [182]:
x_train.shape
# tfidf_matrix.toarray().shape

(40000, 300)

In [183]:

tfidf_matrix = tfidf.transform(newcor[36000:])
# print(tfidf_matrix)
x_test = svd.transform(tfidf_matrix)

tfidf_matrix = tfidf.transform(test)
x_test_r = svd.transform(tfidf_matrix)
y_test = []

for i in range(36000,40000):
    y_test.append(train[i][1])
y_test = np.array(y_test)

# predictions = classifier.predict(x_test)


In [184]:
# x_train.mean()
# predictions
# f1_score(y_test, predictions, average='macro')
x_train.shape

(40000, 300)

In [138]:
predictions = linearClassifier.predict(x_test)
predictions
f1_score(y_test, predictions, average='macro')

/Users/ekkalakleelasornchai/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.18891432308698497

In [223]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import metrics
from sklearn.preprocessing import LabelEncoder

from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))
rate = 0.3

def get_feedforward_nn():
  input1 = Input(shape=(300,))
  x = Dense(100, activation='relu')(input1)
  x = Dropout(0.5)(x)
  x = Dense(10, activation='relu')(x)
  x = Dropout(0.2)(x)  
  out = Dense(1, activation='relu')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='mean_squared_error',
                metrics=[f1])
  return model

In [224]:
from keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from keras import backend as K
K.clear_session()

# Path to save model parameters
weight_path_feedforward_nn='~/workspace/model/model_weight_feedforward_nn2.h5'

# Training callbacks list. TensorBoard() write logs for tensorboard GUI. 
# ModelCheckpoint() writes the resulting model.
# Note that writing to disk takes time (longer than model training time). 
# For other sections, you might not writing any files to disk 
# or write only the graph for TensorBoard.
callbacks_list_feedforward_nn = [
        TensorBoard(log_dir='~/workspace/model/Graph/ff2', histogram_freq=1, write_graph=True, write_grads=True),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

print(x_test.shape)
verbose = 1
model_feedforward_nn = get_feedforward_nn()
train_params = [(20, 32)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_feedforward_nn.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_test, y_test))

(4000, 300)
train with 20 epochs and 32 batch size
Train on 40000 samples, validate on 4000 samples
Epoch 1/20
40000/40000 [==============================] - 4s - loss: 1.7702 - f1: nan - val_loss: 0.5567 - val_f1: 1.0000
Epoch 2/20
40000/40000 [==============================] - 4s - loss: 1.0284 - f1: 1.0000 - val_loss: 0.5210 - val_f1: 1.0000
Epoch 3/20
40000/40000 [==============================] - 4s - loss: 0.8619 - f1: 1.0000 - val_loss: 0.5173 - val_f1: 1.0000
Epoch 4/20
40000/40000 [==============================] - 4s - loss: 0.7223 - f1: 1.0000 - val_loss: 0.4915 - val_f1: 1.0000
Epoch 5/20
40000/40000 [==============================] - 4s - loss: 0.6388 - f1: 1.0000 - val_loss: 0.4806 - val_f1: 1.0000
Epoch 6/20
40000/40000 [==============================] - 4s - loss: 0.5732 - f1: 1.0000 - val_loss: 0.4608 - val_f1: 1.0000
Epoch 7/20
40000/40000 [==============================] - 4s - loss: 0.5337 - f1: 1.0000 - val_loss: 0.4571 - val_f1: 1.0000
Epoch 8/20
40000/40000 [====

RuntimeError: Unable to flush file's cached information (file write failed: time = Sat Mar 10 20:27:42 2018
, filename = '~/workspace/model/model_weight_feedforward_nn2.h5', file descriptor = 45, errno = 28, error message = 'No space left on device', buf = 0xe135110, total write size = 3152, bytes this sub-write = 3152, bytes actually written = 18446744073709551615, offset = 139264)

In [225]:
# from keras.layers import Dropout

# def get_nn_with_dropout():

#     rate = 0.1
    
#     input1 = Input(shape=(21,))
#     x = Dense(100, activation='relu')(input1)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     out = Dense(1, activation='sigmoid')(x)

#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                 loss='binary_crossentropy',
#                 metrics=['acc'])
#     return model

from sklearn.metrics import f1_score,precision_score,recall_score

################################################################################
# Write a function to evaluate your model. Your function must make prediction  #
# using the input model and return f-score, precision, and recall of the model.#
# You can make predictions by calling model.predict().                         #
################################################################################
def evaluate(x_test, y_test, model):
    """
    Evaluate model on the splitted 10 percent testing set.
    """
    y_pred = model.predict(x_test)

#     #map probability to class
#     prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
#     y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    y_pred = np.rint(y_pred).astype(int) 
    print(y_pred.T[0][300:400])
    print(y_test[300:400])
    f1score = f1_score(y_test,y_pred.T[0], average='micro')
#     precision = precision_score(y_test,y_pred,average='macro')
#     recall = recall_score(y_test,y_pred,average='macro')
    return f1score

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [227]:
# print(x_test.shape)
evaluate(x_test, y_test, model_feedforward_nn)

[4 4 3 4 4 4 4 4 3 4 4 4 4 3 3 4 4 4 4 3 3 3 4 4 4 4 4 4 4 3 3 4 4 4 4 3 2
 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 3 5 4 4 3 4 4 4 4 4 2 4 3 4 3 4 4 4 4 4
 3 4 2 4 4 4 4 4 3 3 4 4 4 4 3 4 4 4 4 4 4 3 4 4 4 3]
[3 4 3 4 4 4 5 4 4 3 4 4 5 4 2 3 5 5 4 2 3 4 4 3 4 5 4 4 4 3 4 4 4 3 4 2 1
 3 4 4 3 5 2 4 3 5 5 4 4 5 5 3 4 4 4 5 3 5 4 4 3 3 3 3 3 4 2 3 3 4 4 5 4 4
 3 4 1 3 4 4 5 5 3 3 5 3 4 4 3 3 4 4 4 3 5 3 3 4 5 4]


0.59599999999999997

In [228]:
y_res = model_feedforward_nn.predict(x_test_r)

#     #map probability to class
#     prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
#     y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
y_res = np.rint(y_res).astype(int) 

In [229]:
res = y_res.T[0]
res

array([4, 3, 3, ..., 4, 4, 3])

In [230]:
df_res = pd.DataFrame({"reviewID": np.arange(1,len(res)+1),"rating": res}, columns=["reviewID", "rating"])
df_res.head(10)

,reviewID,rating
0,1,4
1,2,3
2,3,3
3,4,4
4,5,4
5,6,4
6,7,4
7,8,3
8,9,4
9,10,4


In [232]:
df_res.to_csv("result.csv", index=False)